In [77]:
from datetime import datetime
datetime.now(tz=None)

datetime.datetime(2020, 8, 22, 13, 45, 29, 835634)

In [80]:
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

engine = create_engine('sqlite:///hospital_last.db')
meta = MetaData()

Hospital = Table('Hospital',meta,
Column('dutyAddr', String),
Column('Add_level1', String),
Column('Add_level2', String),
Column('Add_level3', String),
Column('dutyDiv', String),
Column('dutyDivNam', String),
Column('dutyEmcls', String),
Column('dutyEmclsName', String),
Column('dutyEryn', String),
Column('dutyMapimg', String),
Column('dutyName', String),
Column('dutyTel1', String),
Column('dutyTime1c', String),
Column('dutyTime1s', String),
Column('dutyTime2c', String),
Column('dutyTime2s', String),
Column('dutyTime3c', String),
Column('dutyTime3s', String),
Column('dutyTime4c', String),
Column('dutyTime4s', String),
Column('dutyTime5c', String),
Column('dutyTime5s', String),
Column('dutyTime6c', String),
Column('dutyTime6s', String),
Column('hpid', String),
Column('postCdn1', String),
Column('postCdn2', String),
Column('rnum', String, primary_key=True),
Column('wgs84Lat', String),
Column('wgs84Lon', String))
meta.create_all(engine)

In [34]:
result[0]['dutyAddr'].split()

['경기도', '여주시', '세종로', '11']

In [81]:
import xmltodict
import json
import requests

for i in range(1,200):
    a = "http://apis.data.go.kr/B552657/HsptlAsembySearchService/getHsptlMdcncFullDown?pageNo="+str(i)+"&numOfRows=500&ServiceKey=jPNWIruN1H52BtrIdrslPWPBfN2mY28AXvsD%2BscxfWuaeVoclCVyrtokg5YYTJVSEKJil0kdBpsXX5N%2BnB5cKA%3D%3D"
    response = requests.get(a)
    cc = xmltodict.parse(response.text)
    dd = json.loads(json.dumps(cc))
    result = dd['response']['body']['items']['item']
    
    
    for j in range(500):
        tmp = result[j]['dutyAddr'].split()

        result[j]['Add_level1'] = tmp[0]
        result[j]['Add_level2'] = tmp[1]
        result[j]['Add_level3'] = tmp[2]
        engine.execute(Hospital.insert(), result[j])

KeyError: 'response'

In [115]:
Location( 126.95391488470001, 37.48337892720001)

('서울특별시', '관악구', '관악로')

In [146]:
len(tmp)

75

In [143]:
tmp[0][5], tmp[0][6]

('37.479738442', '126.9528769771')

# 현재 위치 좌표 검색 후 주변 병원 탐색

In [ ]:
import os 
import numpy as np 
import folium
from folium import plugins

In [112]:
def Location(Lon,Lat):
    geoUrl = 'http://api.vworld.kr/req/address?service=address&request=getAddress&version=2.0&crs=epsg:4326&point=' + str(Lon) + ',' + str(Lat) + '&format=xml&type=road&zipcode=true&simple=false&key=35C83A30-300F-3C4C-9224-D8712B43726D'
    response = requests.get(geoUrl)
    xmldict = xmltodict.parse(response.text)
    geoResult = json.loads(json.dumps(xmldict))
    return geoResult['response']['result']['item']['structure']['level1'], geoResult['response']['result']['item']['structure']['level2'], geoResult['response']['result']['item']['structure']['level4L']

In [123]:
# https://www.zerocho.com/category/HTML&DOM/post/59155228a22a5d001827ea5d < 현재 위치 정보 가지고 오기
#Location('37.649400290444504', '127.08134511742941')
Location(127.08134511742941,37.649400290444504)

('서울특별시', '노원구', '중계로')

In [134]:
m = folium.Map([37.649400290444504,127.08134511742941], zoom_start=20)
folium.Marker([37.5665, 126.9780]).add_to(m)

In [153]:
import numbers
import math

class GeoUtil:
    """
    Geographical Utils
    """
    @staticmethod
    def degree2radius(degree):
        return degree * (math.pi/180)
    
    @staticmethod
    def get_harversion_distance(x1, y1, x2, y2, round_decimal_digits=5):
        """
        경위도 (x1,y1)과 (x2,y2) 점의 거리를 반환
        Harversion Formula 이용하여 2개의 경위도간 거리를 구함(단위:Km)
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        R = 6371 # 지구의 반경(단위: km)
        dLon = GeoUtil.degree2radius(x2-x1)    
        dLat = GeoUtil.degree2radius(y2-y1)

        a = math.sin(dLat/2) * math.sin(dLat/2) \
            + (math.cos(GeoUtil.degree2radius(y1)) \
              *math.cos(GeoUtil.degree2radius(y2)) \
              *math.sin(dLon/2) * math.sin(dLon/2))
        b = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        return round(R * b, round_decimal_digits)

In [168]:
tmp = Search_loc("'서울특별시'", "'노원구'", "'중계로'")
lat, lon = 37.64928136787053,127.08133592498548
m = folium.Map([lat, lon], zoom_start=20)

for i in range(len(tmp)):
    print(tmp[i][5], tmp[i][6])
    if GeoUtil.get_harversion_distance(lon, lat, float(tmp[i][6]), float(tmp[i][5])) < 0.5:
        folium.Marker([tmp[i][5], tmp[i][6]]).add_to(m)
    print(tmp[i][5], tmp[i][6])
m

37.6508417136906 127.077525686582
37.6508417136906 127.077525686582
37.65055008500393 127.0755898660955
37.65055008500393 127.0755898660955
37.6508513558 127.076000097
37.6508513558 127.076000097
37.64925261327217 127.08122257961588
37.64925261327217 127.08122257961588
37.6503834948 127.07498429769998
37.6503834948 127.07498429769998
37.65040990136575 127.07500048179897
37.65040990136575 127.07500048179897
37.6505017776 127.0759952081
37.6505017776 127.0759952081
37.649400290444504 127.08134511742941
37.649400290444504 127.08134511742941
37.64928136787053 127.08133592498548
37.64928136787053 127.08133592498548


# 위치 기반 DB 검색

In [121]:
def Search_loc(level1, level2, level4L):
    conn = sqlite3.connect('hospital_last.db')
    cur = conn.cursor()
    cur.execute('''
    SELECT dutyName, dutyTel1, dutyDivNam, dutyAddr, dutyMapimg,wgs84Lat, wgs84Lon 
    FROM Hospital as H
    WHERE H.Add_level1 = ''' + level1 +'''
    and H.Add_level2 = ''' + level2 +'''
    and H.Add_level3 = ''' + level4L)
    res = cur.fetchall()
    
    return res

In [167]:
Search_loc("'서울특별시'","'노원구'","'중계로'")

[('중계은행사거리마취통증의학과의원',
  '02-931-7578',
  '의원',
  '서울특별시 노원구 중계로 206, 세일학원 2층 (중계동)',
  None,
  '37.6508417136906',
  '127.077525686582'),
 ('서울뜻이룸치과의원',
  '02-952-1004',
  '치과의원',
  '서울특별시 노원구 중계로 219',
  None,
  '37.65055008500393',
  '127.0755898660955'),
 ('럼플란트치과의원',
  '02-3391-2004',
  '치과의원',
  '서울특별시 노원구 중계로 218, 801호 (중계동)',
  None,
  '37.6508513558',
  '127.076000097'),
 ('파티마의원',
  '02-933-0081',
  '의원',
  '서울특별시 노원구 중계로 160, 화인상가아파트 (중계동)',
  '수암초등학교 옆 화인상가',
  '37.64925261327217',
  '127.08122257961588'),
 ('은행나무한의원',
  '02-3392-5911',
  '한의원',
  '서울특별시 노원구 중계로 225, 4층 (중계동, 청구3차아파트)',
  None,
  '37.6503834948',
  '127.07498429769998'),
 ('이튼치과의원',
  '02-934-2872',
  '치과의원',
  '서울특별시 노원구 중계로 225',
  '상계역, 중계역에서 버스로 이동/',
  '37.65040990136575',
  '127.07500048179897'),
 ('연세스마일치과의원',
  '02-931-2666',
  '치과의원',
  '서울특별시 노원구 중계로 217, 4층 (중계동, 중계중흥빌딩)',
  '중계동 학원가 은행사거리 인근',
  '37.6505017776',
  '127.0759952081'),
 ('예담치과의원',
  '02-935-7528',
  '치과의원',
  '서울특별시 노원구 중계로 160',
  